# Embedded CFG Experiment `simple_timer`

Testing the embedded FXE implementation on `simple_timer` example binaries from the nRF5 SDK (v17.0.2)

In [1]:
import os
from copy import deepcopy

from ffxe import *

In [2]:
%%time
ffxe = FFXEngine(
    pd="mmaps/nrf52832.yml",
    path="examples/simple_timer.elf",
    log_stdout=True,
    log_insn=True,
)
ffxe.run()

2023-01-01 17:08:21,919 : input path: examples/simple_timer.elf
2023-01-01 17:08:21,921 : backup context @ 0x2b1
2023-01-01 17:08:21,921 : backup context @ 0x2b1
2023-01-01 17:08:21,921 : backup context @ 0x2b1
2023-01-01 17:08:21,921 : backup context @ 0x2b1
2023-01-01 17:08:21,922 : backup context @ 0x2b1
2023-01-01 17:08:21,922 : backup context @ 0x2b1
2023-01-01 17:08:21,922 : backup context @ 0x2b1
2023-01-01 17:08:21,923 : backup context @ 0x2b1
2023-01-01 17:08:21,923 : backup context @ 0x2b1
2023-01-01 17:08:21,923 : backup context @ 0x2b1
2023-01-01 17:08:21,924 : backup context @ 0x2b1
2023-01-01 17:08:21,924 : backup context @ 0x2b1
2023-01-01 17:08:21,924 : backup context @ 0x2b1
2023-01-01 17:08:21,924 : backup context @ 0x2b1
2023-01-01 17:08:21,925 : backup context @ 0x2b1
2023-01-01 17:08:21,925 : backup context @ 0x2b1
2023-01-01 17:08:21,925 : backup context @ 0x2b1
2023-01-01 17:08:21,926 : backup context @ 0x2b1
2023-01-01 17:08:21,926 : backup context @ 0x2b1
2023-

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:
total_insns = 0
total_size = 0
for block in ffxe.cfg.bblocks.values():
    total_size += block.size
    for insn in ffxe.cs.disasm(
            ffxe.uc.mem_read(block.addr, block.size), block.size):
        total_insns += 1
print(f"Found {len(ffxe.cfg.bblocks)} blocks, "
      f"{len(ffxe.cfg.edges)} edges, "
      f"{total_insns} insns, "
      f"{hex(total_size)} bytes, "
      "\n")

Found 329 blocks, 351 edges, 1294 insns, 0xe2e bytes, 



bug screwing up function registration:
an invalid edge is causing registration function to execute with invalid context, so since it is already explored when the function is called in a correct context, the block quota is already depleted, and the function doesn't get properly registered.

can fix this by disallowing execution of the start of a function block (defined by the push/stmdb instruction) if was not preceded by a call instruction. (call instructions will always put the function on the callstack, so we can check the call stack when we encounter a push/stmdb inst to double check that it was indeed called.

this assumes that push/stmdb must occur in the first basic block of the function, and is not used in other blocks of the function (which is probably against convention anyways, though i need to double check this)

bad assumption. that didn't work. there's a function that doesn't have push in the first block
there are calls that don't get put on the callstack for some reason too.

2nd try: if another call encounters an existing block that doesn't have the same function, increment its quota and update its calling context

In [4]:
disasm_txt = deepcopy(ffxe.fw.disasm_txt)
for block in ffxe.cfg.bblocks.values():
    insn_addr = block.addr
    for insn in ffxe.cs.disasm(
            ffxe.uc.mem_read(block.addr, block.size), block.size):
        try:
            lineno = ffxe.fw.disasm[insn_addr]['line']
            insn_addr += insn.size
            disasm_txt[lineno] = "{}{}{}{:<100}\u001b[0m".format(
                "\u001b[103m",  # bright yellow background
                "\u001b[30m",   # black foreground
                "\u001b[1m",    # bold font
                disasm_txt[lineno])
        except Exception:
            pass
print('\n'.join(disasm_txt))


examples/simple_timer.elf:     file format elf32-littlearm


Disassembly of section .text:

00000000 <__isr_vector>:
       0:	20010000 	.word	0x20010000
       4:	000002b1 	.word	0x000002b1
       8:	000002d9 	.word	0x000002d9
       c:	000002db 	.word	0x000002db
      10:	000002dd 	.word	0x000002dd
      14:	000002df 	.word	0x000002df
      18:	000002e1 	.word	0x000002e1
	...
      2c:	000002e3 	.word	0x000002e3
      30:	000002e5 	.word	0x000002e5
      34:	00000000 	.word	0x00000000
      38:	000002e7 	.word	0x000002e7
      3c:	000002e9 	.word	0x000002e9
      40:	000002eb 	.word	0x000002eb
      44:	000002eb 	.word	0x000002eb
      48:	000002eb 	.word	0x000002eb
      4c:	000002eb 	.word	0x000002eb
      50:	000002eb 	.word	0x000002eb
      54:	000002eb 	.word	0x000002eb
      58:	00000515 	.word	0x00000515
      5c:	000002eb 	.word	0x000002eb
      60:	000002eb 	.word	0x000002eb
      64:	00000af5 	.word	0x00000af5
      68:	000002eb 	.word	0x000002eb
      6c:	000002eb 	.word	